In [15]:
from order_book import Book
from order_book import Format
from event import Event
from datetime import datetime
import pandas as pd
import numpy as np

# set up logger to track events
import log
logger = log.get_logger('Main')
logger.info('testing logger from module Main')

# import lobster message file
cols = ['time', 'type', 'id', 'shares', 'price', 'direction']
data = pd.read_csv("data/lobster/AAPL_2012-06-21_34200000_37800000_message_50.csv", names=cols)
# re-scale the price col
data.price = data.price/10000
# make sure data is during market hours
data = data[data['time']>= 9.5*60*60]
data = data[data['time']<= 16*60*60]
print(len(data))
data.head()

91997


,time,type,id,shares,price,direction
0,34200.004241,1,16113575,18,585.33,1
1,34200.004261,1,16113584,18,585.32,1
2,34200.004447,1,16113594,18,585.31,1
3,34200.025552,1,16120456,18,585.91,-1
4,34200.025580,1,16120480,18,585.92,-1


In [16]:
# create order book object
book = Book()
events = []

# iterate through event messages and read into book - test using first 20k events
for i in range(20000):
    event = Event(data.loc[i])
    book.handleEvent(event, i)
    events.append(event)

In [17]:
# pull all the visible executions into a trades data frame
# split them into buys and sells based on direction
trades = pd.DataFrame(book.trades, columns=['Time', 'Price', 'Shares', 'Direction'])
trades = trades.set_index(['Time'])
sells = trades[trades['Direction']==-1]
buys = trades[trades['Direction']==1]

In [18]:
# pull all the hidden executions into a trades data frame
# split them into buys and sells based on direction
hidden_trades = pd.DataFrame(book.hidden_trades, columns=['Time', 'Price', 'Shares', 'Direction'])
hidden_trades = hidden_trades.set_index(['Time'])
hidden_sells = hidden_trades[hidden_trades['Direction']==-1]
hidden_buys = hidden_trades[hidden_trades['Direction']==1]

In [19]:
# get the book snapshots list and format it to match the LOBSTER output
# We split the bid and ask side of the book to make it easier to plot below
# remove the first few rows as they are 0
fmtbk = Format(book.book_snapshot)
b = fmtbk.formatBook()
a = b.copy()
bids = b.drop(columns=['Ask_1','Ask_1_Vol', 'Ask_2','Ask_2_Vol','Ask_3','Ask_3_Vol','Ask_4','Ask_4_Vol','Ask_5','Ask_5_Vol'])
asks = a.drop(columns=['Bid_1', 'Bid_1_Vol', 'Bid_2', 'Bid_2_Vol', 'Bid_3', 'Bid_3_Vol', 'Bid_4', 'Bid_4_Vol', 'Bid_5', 'Bid_5_Vol'])
bids = bids.iloc[5:]
asks = asks.iloc[5:]
bids.head()

,Bid_1,Bid_1_Vol,Bid_2,Bid_2_Vol,Bid_3,Bid_3_Vol,Bid_4,Bid_4_Vol,Bid_5,Bid_5_Vol
34200.025613,585.33,18.0,585.32,18.0,585.31,18.0,0.0,0.0,0.0,0.0
34200.050241,585.33,18.0,585.32,18.0,585.31,18.0,585.0,100.0,0.0,0.0
34200.074199,585.33,18.0,585.32,18.0,585.31,18.0,585.0,100.0,0.0,0.0
34200.074256,585.33,18.0,585.32,18.0,585.31,18.0,585.0,100.0,0.0,0.0
34200.074293,585.33,18.0,585.32,18.0,585.31,18.0,585.0,100.0,0.0,0.0


In [20]:
import plotly.graph_objects as go
import plotly.io as pio
# set up all line traces for different features of our data set
# Scatter plots for sells and buys
sell_trace = go.Scatter(x=sells.index, y=sells.Price, mode='markers',
                        marker=dict(color='Green', size=sells.Shares/10),
                        name='Sell',
                        text = 'Price:  ' + sells['Price'].astype(str) + '<br>' + 'Volume: ' + sells['Shares'].astype(str))
buy_trace = go.Scatter(x=buys.index, y=buys.Price, mode='markers',
                       marker=dict(color='Red', size=buys.Shares/10),
                       name='Buy',
                       text = 'Price: ' + buys['Price'].astype(str) + '<br>' + 'Volume: ' + buys['Shares'].astype(str))

# Scatter plosts for hidden sells and buys
hidden_sell_trace = go.Scatter(x=hidden_sells.index, y=hidden_sells.Price, mode='markers',
                        marker=dict(color='Yellow', size=sells.Shares/10),
                        name='Hidden Sell',
                        text = 'Price:  ' + hidden_sells['Price'].astype(str) + '<br>' + 'Volume: ' + hidden_sells['Shares'].astype(str))
hidden_buy_trace = go.Scatter(x=hidden_buys.index, y=hidden_buys.Price, mode='markers',
                       marker=dict(color='Yellow', size=hidden_buys.Shares/10),
                       name='Hidden Buy',
                       text = 'Price: ' + hidden_buys['Price'].astype(str) + '<br>' + 'Volume: ' + hidden_buys['Shares'].astype(str))

# Line plot for trades
trade_trace = go.Scatter(x=trades.index, y=trades.Price, mode='lines',
                         line=dict(color='blue'),
                         name='Trades')

# Line plots for asks and bids
ask_line_trace = go.Scatter(x=asks.index, y=asks.Ask_1, mode='lines',
                            line=dict(color='rgba(255, 0, 0, 1)'),
                            name='Ask Line',
                            text = 'Price:  ' + asks['Ask_1'].astype(str) + '<br>' + 'Volume: ' + asks['Ask_1_Vol'].astype(str))

bid_line_trace = go.Scatter(x=bids.index, y=bids.Bid_1, mode='lines',
                            line=dict(color='rgba(0, 255, 0, 1)'),
                            name='Bid Line',
                            text = 'Price:  ' + bids['Bid_1'].astype(str) + '<br>' + 'Volume: ' + bids['Bid_1_Vol'].astype(str))


In [21]:
graph_execution_data = [sell_trace, buy_trace, hidden_sell_trace, hidden_buy_trace, trade_trace]

# Configure layout
layout = go.Layout(title='AAPL Executions',
                   xaxis=dict(title='Time'),
                   yaxis=dict(title='Price'),
                   height=750)

# Create figure
fig = go.Figure(data=graph_execution_data, layout=layout)
fig.write_html("data/html_files/AAPL_Executions.html")
#pio.write_image(fig, 'AAPL_Executions.png', format='png', scale=1, width=1400, height=800)
#fig.show()

![AAPL Executions](data/images/AAPL_Executions.png)

In [22]:

graph_bbo_data = [ask_line_trace, bid_line_trace]

# Configure layout
layout = go.Layout(title='AAPL BBO',
                   xaxis=dict(title='Time'),
                   yaxis=dict(title='Price'),
                   height=750)

# Create figure
fig = go.Figure(data=graph_bbo_data, layout=layout)
fig.write_html("data/html_files/AAPL_BBO.html")
#pio.write_image(fig, 'AAPL_BBO.png', format='png', scale=1, width=1400, height=800)
#fig.show()

![AAPL BBO](data/images/AAPL_BBO.png)

In [23]:
# compare to LOBSTER Orderbook file
data = pd.read_csv("data/lobster/AAPL_2012-06-21_34200000_57600000_orderbook_1.csv", names=['Ask_1', 'Ask_1_Vol', 'Bid_1', 'Bid_1_Vol'])
# trip to roughly the same time frame
data = data[:10000]
bids = data.drop(columns=['Ask_1', 'Ask_1_Vol'])
asks = data.drop(columns=['Bid_1', 'Bid_1_Vol'])

# Line plots for asks and bids
ask_line_trace = go.Scatter(x=asks.index, y=asks.Ask_1, mode='lines',
                            line=dict(color='rgba(255, 0, 0, 1)'),
                            name='Ask Line',
                            text = 'Price:  ' + asks['Ask_1'].astype(str) + '<br>' + 'Volume: ' + asks['Ask_1_Vol'].astype(str))


bid_line_trace = go.Scatter(x=bids.index, y=bids.Bid_1, mode='lines',
                            line=dict(color='rgba(0, 255, 0, 1)'),
                            name='Bid Line',
                            text = 'Price:  ' + bids['Bid_1'].astype(str) + '<br>' + 'Volume: ' + bids['Bid_1_Vol'].astype(str))

graph_data = [ask_line_trace, bid_line_trace]

# Configure layout
layout = go.Layout(title='AAPL BBO LOBSTER',
                   xaxis=dict(title='Time'),
                   yaxis=dict(title='Price'),
                   height=750)

# Create figure
fig = go.Figure(data=graph_data, layout=layout)
fig.write_html("data/html_files/AAPL_BBO_LOBSTER.html")
#pio.write_image(fig, 'AAPL_BBO_LOBSTER.png', format='png', scale=1, width=1400, height=800)
#fig.show()

![AAPL BBO LOBSTER](data/images/AAPL_BBO_LOBSTER.png)